In [1]:
# Import needed MASTML modules

from mastml.mastml import Mastml
from mastml.datasets import LocalDatasets, SklearnDatasets, FoundryDatasets, FigshareDatasets, DataUtilities, DataCleaning
from mastml.preprocessing import SklearnPreprocessor, MeanStdevScaler
from mastml.models import SklearnModel
from mastml.data_splitters import SklearnDataSplitter, NoSplit
from mastml.plots import Histogram, Scatter
from mastml.feature_selectors import EnsembleModelFeatureSelector, NoSelect, PearsonSelector, MASTMLFeatureSelector
from mastml.feature_generators import ElementalFeatureGenerator

To install latest forestci compatabilty with scikit-learn>=0.24, run pip install git+git://github.com/scikit-learn-contrib/forest-confidence-interval.git
To import data from figshare, manually install figshare via git clone of git clone https://github.com/cognoma/figshare.git
If you want to use XGBoost models, please manually install xgboost package with pip install xgboost. If have error with finding libxgboost.dylib library, dobrew install libomp. If do not have brew on your system, first do ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)" from the Terminal


In [19]:
# Initialize the MASTML run, write savepath and metadata file

mastml = Mastml(savepath='results/test_output')
savepath = mastml.get_savepath
mastml_metadata = mastml.get_mastml_metadata

results/test_output not empty. Renaming...


In [20]:
# Download diffusion data from Figshare (done previously)

#article_id = 7418492
#FigshareDatasets().download_data(article_id=article_id)

In [21]:
# Load in diffusion data downloaded from Figshare 

target = 'E_regression.1'
extra_columns = ['E_regression', 'Material compositions 1', 'Material compositions 2', 'Hop activation barrier']
d = LocalDatasets(file_path='figshare_7418492/All_Model_Data_Missing.xlsx', 
                  target=target, 
                  extra_columns=extra_columns, 
                  group_column='Material compositions 1',
                  as_frame=True)

X, y, groups = d.load_data()


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [22]:
X

NameError: name 'X' is not defined

In [23]:
groups

NameError: name 'groups' is not defined

In [24]:
# The data contains missing values. Clean the data, which corrects missing values
# and provides some basic analysis of the input data

cleaner = DataCleaning()
X, y = cleaner.evaluate(X=X, 
                        y=y, 
                        method='imputation', 
                        strategy='mean', 
                        savepath=savepath)

NameError: name 'X' is not defined

In [25]:
X

NameError: name 'X' is not defined

In [26]:
# Preprocess the cleaned data to be normalized

preprocessor = SklearnPreprocessor(preprocessor='StandardScaler', as_frame=True)
X = preprocessor.evaluate(X=X, 
                          savepath=savepath)

#preprocessor = MeanStdevScaler(mean=0, stdev=2, as_frame=True)
#X = preprocessor.evaluate(X=X,
#                         savepath=savepath)

NameError: name 'X' is not defined

In [27]:
#print(X['valence_composition_average'].mean(), X['valence_composition_average'].std())

In [28]:
#X

In [29]:
# Define two models and two feature selector types to perform

#model1 = SklearnModel(model='KernelRidge', kernel='rbf')
model2 = SklearnModel(model='RandomForestRegressor')
#model2 = SklearnModel(model='LinearRegression')
#models = [model1, model2]
models = [model2]

selector1 = NoSelect()
selector2 = EnsembleModelFeatureSelector(model=SklearnModel(model='RandomForestRegressor'), n_features_to_select=10)
selector3 = PearsonSelector(threshold_between_features=0.9, threshold_with_target=0.7, flag_highly_correlated_features=True, n_features_to_select=15)
selector4 = MASTMLFeatureSelector(model=SklearnModel(model='LinearRegression'), n_features_to_select=1)
#selectors = [selector1, selector2]
selectors = [selector4]

metrics = ['r2_score', 'mean_absolute_error', 'root_mean_squared_error', 'rmse_over_stdev']

plots = ['Scatter']

In [30]:
# Define and run the case where no data split is performed (full fit)
#
#splitter = NoSplit()
#splitter.evaluate(X=X,
#                  y=y, 
#                  models=models,
#                  selectors=selectors,
#                  metrics=metrics,
#                  savepath=savepath)

In [31]:
# Define and run the case where random RepeatedKFold CV is performed

splitter = SklearnDataSplitter(splitter='RepeatedKFold', n_repeats=2, n_splits=5)
splitter.evaluate(X=X,
                  y=y, 
                  models=models,
                  selectors=selectors,
                  metrics=metrics,
                  plots=plots,
                  savepath=savepath)

NameError: name 'X' is not defined

In [32]:
# Define and run the case where LeaveOutGroup CV is performed

splitter = SklearnDataSplitter(splitter='LeaveOneGroupOut')
splitter.evaluate(X=X,
                  y=y, 
                  groups=groups,
                  models=models,
                  selectors=selectors,
                  metrics=metrics,
                  savepath=savepath)

NameError: name 'X' is not defined

In [33]:
import sklearn, pandas, numpy, matplotlib, pymatgen, matminer
print(sklearn.__version__, pandas.__version__, numpy.__version__, matplotlib.__version__, matminer.__version__, pymatgen.__version__)

ModuleNotFoundError: No module named 'pymatgen'

In [34]:
!which python

'which' is not recognized as an internal or external command,
operable program or batch file.


In [35]:
import pandas as pd
d = {"mae": 1, "rmse": 0.5}
df = pd.DataFrame(d, index=[0])
df

,mae,rmse
0,1,0.5


In [ ]:
df.to_excel('test.xlsx', index=None)

In [ ]:
df_import = pd.read_excel('test.xlsx')
d = df_import.to_dict('records')[0]
print(df_import)
print(d)

In [ ]:
import numpy as np
yt = np.array([1, 1, 2, 3])
yp = np.array([1.2, 0.9, 2, 3])
y_unique = np.unique(yt)
res = abs(yp-yt)
print(res)
for y in list(y_unique):
    print(y)
    print(yp[np.where(yt==y)])
    best = min(abs(yp[np.where(yt==y)]-yt[np.where(yt==y)]))
    print(best)
    print(yp[np.where(res==best)])

In [ ]:
np.where(yt<2)

In [ ]:
a = [1, 2, 3]
b = [4, 5, 6]
c = [7, 8, 9]
d = [10, 11, 12]
for (i1, i2), (j1, j2) in zip(zip(a, b), zip(c, d)):
    print(i1, i2, j1, j2)